In [1]:
"""
Simple tester for the vgg19_trainable
"""
import numpy as np
import tensorflow as tf
import vgg19_trainable as vgg19
import cv2
import utils

img1 = cv2.imread("./test_data/xray.png")
img1 = cv2.cvtColor(img1,cv2.COLOR_BGR2RGB)

norm_img = cv2.normalize(img1,None,alpha=0.0, beta=1.0,norm_type=cv2.NORM_MINMAX,dtype=cv2.CV_32FC3)
img1_true_result = np.array([1 if i == 1 else 0 for i in range(2)])  # 1-hot result for tiger

batch1 = norm_img.reshape((1, 64, 64, 3))
ylabel = img1_true_result.reshape((1,1,2))

/opt/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
tf.reset_default_graph()

with tf.device('/cpu:0'):
    sess = tf.Session()

    images = tf.placeholder(tf.float32, [1, 64, 64, 3])
    true_out = tf.placeholder(tf.float32, [1, 1, 2])
    train_mode = tf.placeholder(tf.bool)

    #vgg = vgg19.Vgg19('./test-save.npy')
    vgg = vgg19.Vgg19()
    vgg.build(images, train_mode)

    
    # print number of variables used: 143667240 variables, i.e. ideal size = 548MB
    print(vgg.get_var_count())

    # simple 1-step training
    #cost = tf.reduce_mean(-tf.reduce_sum(ylabel * tf.log(vgg.prob), reduction_indices=1)) 
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=ylabel,logits=vgg.logit)
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    
    train = tf.train.AdamOptimizer(0.005).minimize(cross_entropy_mean)
      
    sess.run(tf.global_variables_initializer())
    error = sess.run(cross_entropy_mean, feed_dict={images: batch1, train_mode: False})
    print(error)    
    
    for i in range(100):
        sess.run(train, feed_dict={images: batch1, true_out:ylabel , train_mode: True})
        
    # test classification again, should have a higher probability about tiger
    error = sess.run(cross_entropy_mean, feed_dict={images: batch1, train_mode: False})
    print(error)
    


14845890
0.6932234
0.0


### input pipe_line

In [ ]:
def parse_img(name)
data = tf.data.Dataset.from_tensor_slices(flist)
data = data.m